In [1]:
import sqlite3
import pandas as pd

In [2]:
sqlite_file = 'data/database.sqlite'

In [3]:
def create_connection(db):
    """ connect to a sqlite database
    :param db: database file
    :return: a sqlite db connection object, 
      none if error
    """
    try:
        conn = sqlite3.connect(db)
        return conn
    except Error as e:
        print(e)
 
    return None

In [4]:
db = create_connection(sqlite_file)

In [5]:
# overview of table schema
display(pd.read_sql_query("PRAGMA table_info('May2015');", db))

# more info about what each field means:
# https://github.com/reddit-archive/reddit/wiki/JSON

,cid,name,type,notnull,dflt_value,pk
0,0,created_utc,INTEGER,0,None,0
1,1,ups,INTEGER,0,None,0
2,2,subreddit_id,,0,None,0
3,3,link_id,,0,None,0
4,4,name,,0,None,0
5,5,score_hidden,,0,None,0
6,6,author_flair_css_class,,0,None,0
7,7,author_flair_text,,0,None,0
8,8,subreddit,,0,None,0
9,9,id,,0,None,0


In [6]:
# sample a few records

sample_rows = "SELECT \
                 subreddit, \
                 author, \
                 ups, \
                 downs, \
                 score, \
                 body \
               FROM \
                 May2015 \
               LIMIT 5;"

display(pd.read_sql_query(sample_rows, db))

,subreddit,author,ups,downs,score,body
0,soccer_jp,rx109,4,0,4,くそ\n読みたいが買ったら負けな気がする\n図書館に出ねーかな
1,nba,WyaOfWade,4,0,4,gg this one's over. off to watch the NFL draft...
2,politics,Wicked_Truth,0,0,0,Are you really implying we return to those tim...
3,AskReddit,jesse9o3,3,0,3,No one has a European accent either because i...
4,AskReddit,beltfedshooter,3,0,3,"That the kid ""..reminds me of Kevin."" so sad..."


In [7]:
# list the top subreddits

top_subreddits = "SELECT \
                    subreddit, \
                    COUNT(*) \
                  FROM \
                    May2015 \
                  GROUP BY 1 \
                  ORDER BY 2 DESC LIMIT 5;"

display(pd.read_sql_query(top_subreddits, db))

,subreddit,COUNT(*)
0,AskReddit,4234970
1,leagueoflegends,1223184
2,nba,756195
3,funny,745916
4,pics,630925


In [8]:
# sample full posts

# do not truncate column
pd.set_option('display.max_colwidth', -1)

body_samples = "SELECT \
                  body \
                FROM \
                  May2015 \
                LIMIT 5;"

display(pd.read_sql_query(body_samples, db))

# back to iPython default
pd.set_option('display.max_colwidth', 50)

,body
0,くそ\n読みたいが買ったら負けな気がする\n図書館に出ねーかな
1,gg this one's over. off to watch the NFL draft I guess
2,"Are you really implying we return to those times or anywhere near that political environment? If so, you won't have much luck selling the American people on that governance concept without ushering in American Revolution 2.0."
3,No one has a European accent either because it doesn't exist. There are accents from Europe but not a European accent.
4,"That the kid ""..reminds me of Kevin."" so sad :-("


In [9]:
def clean_up_sample(df):
    """ data cleanup for reddit sample 
    
    :param df: a pandas data dataframe with
      reddit posts data
    :return clean_df: a cleaned dataframe
    """
    pass
    

In [10]:
def export_sample_to_csv(nrows, table, db, dest = 'data/reddit_extracted_data.csv'):
    """ extract a random sample of rows from
      the reddit comments dataset
    
    :param nrows: number of rows to sample
    :param table: table we will sample from
    :param db: a database connection object
    :param dest: filename for csv export
    """
    
    # get total rows to know the percentage sample that will get us nrows
    total_rows = pd.read_sql_query("SELECT COUNT(*) FROM {};".format(table), db).iloc[0,0]
    
    sample_percentage = nrows/total_rows
    
    sample_sql = "SELECT subreddit, ups, downs, score, body, link_id, id, parent_id, name \
                  FROM May2015 \
                  WHERE ABS(CAST(RANDOM() AS REAL))/9223372036854775808 < {} \
                 ;".format(sample_percentage)
    
    sample_df = pd.read_sql(sample_sql, db)
    
    sample_df.to_csv(dest, index = False)
    
    print(len(sample_df.index))
    
    return(sample_df)
    
#sample2_df = export_sample_to_csv(100, 'May2015', db, 'data/reddit_extracted_data_trying_stuff.csv')

In [11]:
def remove_duplicates(df, existing_indices):
    """ remove duplicates from subsequent 
        sample chunks.
    
    :param df: new dataframe that (presumably)
      contains duplicates
    :param existing_indices: unique identifiers
      already in our sample set.
    :return deduped_df: dataframe with dupes removed
    :return updated_indices: list of indices including
      unique entries from the deduped dataframe
    """

    unique_ids_in_df = set(df.id.unique())
    
    dupe_indices = existing_indices.intersection(unique_ids_in_df)
    print("found {} dupe cases".format(len(dupe_indices)))
    non_dupes = ~df.id.isin(dupe_indices)
    
    # df without dupes
    deduped_df = df[non_dupes]

    # unique ids including new dataframe
    all_unique_ids = existing_indices.union(unique_ids_in_df)
    
    return(deduped_df, all_unique_ids)

sample_df_1 = export_sample_to_csv(10000, 'May2015', db, 'data/reddit_sample_data.csv')
#sample_df_2 = export_sample_to_csv(1000000, 'May2015', db, 'data/reddit_extracted_data_trying_stuff.csv')

#df_2, ex_id = remove_duplicates(sample_df_2, set(sample_df_1.id.unique()))

#print(len(sample_df_2), len(ex_id))

9977


In [100]:
print(len(sample_df_1), len(sample_df_2), len(df_2), len(ex_id))

999661 1001829 983403 1983064


In [17]:
remove_duplicates(df=sample_df)

TypeError: object of type 'NoneType' has no len()